In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

from io import BytesIO #needed for plot
import seaborn as sns; sns.set()

from keras.models import Model
from keras.layers import LSTM, Dropout, Dense, Conv1D, ConvLSTM2D, Input, Bidirectional, TimeDistributed,GRU
from keras.layers import MaxPooling1D, Flatten, BatchNormalization
from keras.utils import plot_model
from keras.models import load_model
from numpy import save
from numpy import load
import sklearn
from scipy import stats
from sklearn import metrics
from keras.optimizers import Adam, SGD, RMSprop
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pickle


In [2]:
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/MyDrive/AI ML Things/University of Turku Research Internship

/content/drive/MyDrive/AI ML Things/University of Turku Research Internship


In [ ]:
cd MyDrive/

/content/drive/MyDrive


In [6]:
ls

 DWT_features/   'Models and Training'/              'Raw Data'/
 MFCC_features/  'Preprocessing and EDA Notebooks'/   WPD_features/


In [7]:
cd MFCC_features/

/content/drive/My Drive/AI ML Things/University of Turku Research Internship/MFCC_features


In [8]:
ls

 Features_According_to_Kaggle/              x_data_reshaped.npy
'Labelled and Unlabelled Data Separated'/   y_data_mfcc.npy
 x_data_mfcc.npy


In [9]:
X = np.load("x_data_mfcc.npy")
y = np.load("y_data_mfcc.npy")
X = X[:583]
y = y[:583]

In [12]:
def one_hot_target(y):
    
    y_one_hot = pd.get_dummies(y)
    
    return y_one_hot.values

In [13]:
y_ = one_hot_target(y)

In [15]:
## FOR CNN model
train_X = X.reshape(-1,  40, 1)
train_y = y_


In [16]:
### FOR LSTM model
train_X = X.reshape(-1,  40, 1)
train_y = y_

In [17]:
### FOR GRU model 
train_X = X.reshape(-1,  40, 1)
train_y = y_

In [18]:
### FOR CNN+LSTM model and CNN + BiLSTM model
train_X = X.reshape(-1,  40, 1)
train_y = y_

In [19]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y ,test_size=0.15, random_state=42, shuffle = True)

In [26]:
def CNN2():

  input_ = Input(shape = ( 40, 1))
  x = (Conv1D(filters = 64, kernel_size = 3, activation = 'relu'))(input_)
  x = (Conv1D(filters =64,kernel_size =  3,activation = 'relu'))(x)
  x = (Dropout(0.1))(x)
  x = (MaxPooling1D(pool_size=2))(x)
  x = (Conv1D(filters =128, kernel_size = 3,activation = 'relu'))(x)
  x = (Conv1D(filters =256,kernel_size = 4,activation ='relu'))(x)
  x = (Dropout(0.2))(x)
  x = (MaxPooling1D(pool_size=2))(x)
  x = (Flatten())(x)
  x = Dense(200, activation = 'relu')(x)
  output = Dense(5, activation = 'softmax')(x)

  model = Model(input_, output)
  
  return model

In [27]:
CNN2()

In [39]:
def LSTM_Model()  : 
    print(y.sum(0))
    input_ = Input(shape = (40,1))
    x = LSTM(40, return_sequences=True)(input_)
    #x = Dropout(0.1)(x)
    x = (LSTM(40, return_sequences= True))(x)
    x = LSTM(40)(x)
    x = Dropout(0.2)(x)
    #x = Dropout(0.2)(x)
    x = Dense(units = 200, activation='relu')(x)
    x = Dense(units = 200, activation='relu')(x)
    x  = Dense(units = 200, activation='relu')(x)
    output = Dense(5,  activation='softmax')(x)

    LSTM_model = Model(input_, output)
    return LSTM_model


In [40]:
LSTM_Model() 

953


In [51]:
def stacked_GRU():

  input_ = Input(shape = (40, 1))
  x = GRU(40, return_sequences =True)(input_)
  x = GRU(40, return_sequences = True)(x)
  x = Dropout(0.2)(x)
  x = GRU(40)(x)
  x = Dense(100, activation = 'relu')(x)
  output = Dense(5, activation = 'softmax')(x)

  GRU_model = Model(input_, output)

  return GRU_model

In [52]:
stacked_GRU()

In [63]:
def CONV_LSTM_model():

    input_ = Input(shape = (40, 1))
    x = (Conv1D(filters=64, kernel_size=3, activation='relu'))(input_)
    x = (Conv1D(filters=64, kernel_size=3, activation='relu'))(x)
    x = (Conv1D(filters=128, kernel_size=3, activation='relu'))(x)
    x = (Dropout(0.2))(x)
    x = (MaxPooling1D(pool_size=2))(x)
    #x = (Flatten())(x)
    x = LSTM(200, return_sequences= True)(x)
    x =LSTM(200, return_sequences=True)(x)
    x = LSTM(200)(x)
    x = Dropout(0.2)(x)
    x = Dense(200, activation='relu')(x)
    output = Dense(5, activation='softmax')(x)

    Conv_LSTM_model = Model(input_, output)

    return Conv_LSTM_model

In [64]:
CONV_LSTM_model()

In [75]:
def CONV_BiLSTM_model():

    input_ = Input(shape = (40, 1))
    x = (Conv1D(filters=64, kernel_size=3, activation='relu'))(input_)
    x = (Conv1D(filters=64, kernel_size=3, activation='relu'))(x)
    x = (Conv1D(filters=128, kernel_size=3, activation='relu'))(x)
    x = (Dropout(0.2))(x)
    x = (MaxPooling1D(pool_size=2))(x)
    #x = TimeDistributed(Flatten())(x)
    x = Bidirectional(LSTM(100, return_sequences= True))(x)
    x = Bidirectional(LSTM(200, return_sequences=True))(x)
    x = LSTM(400)(x)
    x = Dropout(0.2)(x)
    x = Dense(200, activation='relu')(x)
    output = Dense(5, activation='softmax')(x)

    Conv_LSTM_model = Model(input_, output)

    return Conv_LSTM_model

In [76]:
CONV_BiLSTM_model()

In [ ]:
def Conv_LSTM2D_model1():
    input_ = Input(shape = ((40,1, 1, 1)))
    x = ConvLSTM2D(filters = 64, kernel_size = (1,3), activation = 'relu')(input_)
    #x = (Conv1D(filters=64, kernel_size=3, activation='relu'))(x)
    x = (Conv1D(filters=128, kernel_size=3, activation='relu'))(x)
    x = (Dropout(0.2))(x)
    x = (MaxPooling1D(pool_size=2))(x)
    x = TimeDistributed(Flatten())(x)
    x = Bidirectional(LSTM(100, return_sequences= True))(x)
    x = LSTM(200)(x)
    x = Dropout(0.2)(x)
    x = Dense(200, activation='relu')(x)
    output = Dense(5, activation='softmax')(x)
    
    
    ConvLSTM2D_model = Model(input_, output)

    return ConvLSTM2D_model

In [77]:
def train_model_(model):
  
  model  = model ### 
  model.compile(loss = 'categorical_crossentropy', optimizer = Adam(), metrics = ['accuracy'])
        

  r = model.fit(X_train, y_train, batch_size = 32, epochs = 30, validation_split = 0.1)
  #return 4, 6
  return model,r

In [78]:
model, r = train_model_(CONV_BiLSTM_model())

Epoch 1/30
14/14 [==============================] - 7s 125ms/step - loss: 1.2387 - accuracy: 0.5453 - val_loss: 0.9368 - val_accuracy: 0.5800
Epoch 2/30
14/14 [==============================] - 0s 17ms/step - loss: 0.9380 - accuracy: 0.6428 - val_loss: 0.9717 - val_accuracy: 0.6200
Epoch 3/30
14/14 [==============================] - 0s 17ms/step - loss: 0.9854 - accuracy: 0.6339 - val_loss: 0.9146 - val_accuracy: 0.6000
Epoch 4/30
14/14 [==============================] - 0s 16ms/step - loss: 0.8791 - accuracy: 0.6576 - val_loss: 0.8759 - val_accuracy: 0.6800
Epoch 5/30
14/14 [==============================] - 0s 17ms/step - loss: 0.8570 - accuracy: 0.6805 - val_loss: 0.8746 - val_accuracy: 0.6200
Epoch 6/30
14/14 [==============================] - 0s 17ms/step - loss: 0.9148 - accuracy: 0.5994 - val_loss: 0.9186 - val_accuracy: 0.6000
Epoch 7/30
14/14 [==============================] - 0s 17ms/step - loss: 0.8836 - accuracy: 0.6565 - val_loss: 0.8181 - val_accuracy: 0.6600
Epoch 8/30
1

In [79]:

def model_evaluate(model, test_X):
    pred = model.predict(test_X)
    y_te = y_test.argmax(axis = 1)
    pred_te = pred.argmax(axis = 1)
    print("CLASSIFICATION REPORT \n")
    print(classification_report(y_te, pred_te))
    print('\n \n')
    print('CONFUSION MATRIX \n')
    print(confusion_matrix(y_te, pred_te))

In [80]:
## Here the model should be trained on X_train only, use that model to test, F1, Kappa etc..
model_evaluate(model, X_test)

CLASSIFICATION REPORT 

              precision    recall  f1-score   support

           0       1.00      0.71      0.83         7
           1       0.67      0.79      0.73        52
           2       0.00      0.00      0.00         9
           3       0.27      0.43      0.33        14
           4       0.00      0.00      0.00         6

    accuracy                           0.59        88
   macro avg       0.39      0.39      0.38        88
weighted avg       0.52      0.59      0.55        88


 

CONFUSION MATRIX 

[[ 5  1  0  1  0]
 [ 0 41  0 11  0]
 [ 0  7  0  2  0]
 [ 0  8  0  6  0]
 [ 0  4  0  2  0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [81]:
def print_performance_metrics():
  
    print('Accuracy:', np.round(metrics.accuracy_score(y_test, max_y_pred_test),4))
    print('Precision:', np.round(metrics.precision_score(y_test, 
                                max_y_pred_test,average='weighted'),4))
    print('Recall:', np.round(metrics.recall_score(y_test, max_y_pred_test,
                                               average='weighted'),4))
    print('F1 Score:', np.round(metrics.f1_score(y_test, max_y_pred_test,
                                               average='weighted'),4))
    print('Cohen Kappa Score:', np.round(metrics.cohen_kappa_score(y_test.argmax(axis=1), max_y_pred_test.argmax(axis=1)),4))
    print('Matthews Corrcoef:', np.round(metrics.matthews_corrcoef(y_test.argmax(axis=1), max_y_pred_test.argmax(axis=1)),4)) 
    print('ROC AUC:', (metrics.roc_auc_score(y_test, max_y_pred_test,average='macro')))
    print('\t\tClassification Report:\n', metrics.classification_report(y_test, max_y_pred_test))


def print_confusion_matrix_and_save():
    mat = confusion_matrix(y_test.argmax(axis=1), max_y_pred_test.argmax(axis=1))
    sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    #plt.show()

    plt.savefig("Hand.jpg")
    # Save SVG in a fake file object.
    f = BytesIO()
    plt.savefig(f, format="svg")

In [82]:
pred_test   = model.predict(X_test)
max_y_pred_test = np.round(pred_test)
print_performance_metrics()

Accuracy: 0.5795
Precision: 0.5281
Recall: 0.5795
F1 Score: 0.5483
Cohen Kappa Score: 0.2473
Matthews Corrcoef: 0.2552
ROC AUC: 0.6143673893673893
		Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.71      0.83         7
           1       0.69      0.79      0.74        52
           2       0.00      0.00      0.00         9
           3       0.24      0.36      0.29        14
           4       0.00      0.00      0.00         6

   micro avg       0.60      0.58      0.59        88
   macro avg       0.39      0.37      0.37        88
weighted avg       0.53      0.58      0.55        88
 samples avg       0.58      0.58      0.58        88



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [83]:
def KFoldCV(train_X,y, model):  # This performs 10 Fold Cross validation on user dependant model

    scores = []
    train_scores = []
    Kfold_CV = KFold(n_splits = 10, random_state=42)
    i = 0

    for train_index, test_index in Kfold_CV.split(train_X):

        #CNN2()
        i += 1
        print(i)
        
          #break
        Conv_LSTM_model  = model
        Conv_LSTM_model.compile(loss = 'categorical_crossentropy', optimizer = Adam(), metrics = ['accuracy'])
        if i<10:
          print(Conv_LSTM_model.summary())

        r = Conv_LSTM_model.fit(train_X[train_index], y[train_index], batch_size = 32, epochs = 30, validation_data = (train_X[test_index],y[test_index]))

        # Evaluate on train
        pred_train = Conv_LSTM_model.predict(train_X[train_index])
        pred_train_te = pred_train.argmax(axis = 1)
        train_te = y[train_index].argmax(axis = 1)
        accuracy_train   = accuracy_score(train_te, pred_train_te)
        train_scores.append(accuracy_train)
        
        # Evaluate on test
        pred  = Conv_LSTM_model.predict(train_X[test_index])
        pred_te = pred.argmax(axis = 1)
        test_te = y[test_index].argmax(axis = 1)
        accuracy = accuracy_score(test_te, pred_te)
        scores.append(accuracy)

    return scores, train_scores, Conv_LSTM_model,r

In [84]:
### This cell performs 10 Cross Validation , to get Accuracy
## Test Accuracy = np.mean(scores)
## Train Accuracy = np.mean(train_scores)
scores, train_scores, model, r = KFoldCV(train_X,train_y, CONV_BiLSTM_model()) ## Write model type as third parameter

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


1
Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 40, 1)]           0         
_________________________________________________________________
conv1d_39 (Conv1D)           (None, 38, 64)            256       
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 36, 64)            12352     
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 34, 128)           24704     
_________________________________________________________________
dropout_28 (Dropout)         (None, 34, 128)           0         
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 17, 128)           0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 17, 200)           1

In [85]:
np.mean(scores)   ###10Fold Test Sccuracy

0.7270894213909994

In [86]:
np.mean(train_scores) ###10Fold Train Accuracy

0.9238836786623047